## キーワード関連木の作成

### Requirements

In [ ]:
!pip install japanize_matplotlib
!pip install networkx
!pip install matplotlib
!pip install gensim

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/ から
# 20170201.tar.bz2 (2017年2月1日版, 1.3GB, 解凍後 2.6GB) をダウンロードしておく

### 単語埋め込み表現の取得

In [2]:
from gensim.models import KeyedVectors
model_dir = './entity_vector.model.bin'
# 読み込みに時間がかかる
model = KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [136]:
results = model.most_similar('[錦織圭]', topn=5)
for result in results:
    print(result)

('[ロジャー・フェデラー]', 0.8127161264419556)
('[フアン・マルティン・デル・ポトロ]', 0.812140941619873)
('[ノバク・ジョコビッチ]', 0.8057368397712708)
('[ラファエル・ナダル]', 0.8028138279914856)
('[アンディ・マリー]', 0.7997205257415771)


### 幅優先

In [144]:
# jupyterでファイルを開くには File -> Open From Path
# print(plt.__file__)
import networkx as nx
import matplotlib.pyplot as plt
# sns.set()の内部でfont familyをいじられる問題と同様に,nxの内部でいじられる
import japanize_matplotlib
# dequeにより、データの追加を先頭、末尾両方に対してO(1)のコストで実施できる。
# 通常のlistだと先頭に追加する場合にO(n)のコスト。
from collections import deque


keyword = '[集合論]'
G = nx.Graph()
G.add_nodes_from([keyword])


# 幅優先のための queue
queue = deque([keyword])
# 根からの距離
d = {}
d[keyword] = 0
# queueは随時更新されるため、vocabに履歴を保存
vocab = [keyword]
max_depth = 6
while queue:
    parent = queue.popleft()
    if d[parent] >= max_depth:
        break
    children = model.most_similar(parent, topn=5)
    for child, prob in children:
        if child in vocab:
            continue
        vocab.append(child)
        G.add_nodes_from([child])
        G.add_edges_from([(parent, child)])
        queue.append(child)
        d[child] = d[parent] + 1
        

nx.nx_agraph.to_agraph(G).draw(keyword + ".png", prog='fdp', format='png')
print("木構造か？ : ", nx.is_tree(G))
print("ノード数 : ", len(G.nodes()))
print("単一ノード数 : ", len(vocab), len(set(vocab)))

木構造か？ :  True
ノード数 :  49
単一ノード数 :  49 49


### 深さ優先

In [137]:

# 再帰的にノード作成
vocab = []
max_depth = 4
def dfs(parent, depth):
    # 枝刈りできる部分でも most_similar を呼び出しているので幅優先に比べて遅い
    children = model.most_similar(parent, topn=5)
    depth += 1
    if depth >= max_depth:
        return
    for child, prob in children:
        dfs(child, depth)
        if child in vocab:
            continue
        G.add_nodes_from([child])
        G.add_edges_from([(parent, child)])
        vocab.append(child)


# 木の作成、固有名詞などは[]で囲う
# keyword = '集合論'
keyword = '[錦織圭]'
G = nx.Graph()
dfs(keyword, 0)

# ネットワークの可視化
# matplotlib
# nx.draw(G, with_labels=True, font_family="IPAexGothic")
# graphviz
nx.nx_agraph.to_agraph(G).draw(keyword + ".png", prog='fdp', format='png')
print("木構造か？ : ", nx.is_tree(G))
print("ノード数 : ", len(G.nodes()))
print("単一ノード数 : ", len(vocab), len(set(vocab)))

木構造か？ :  True
ノード数 :  21
単一ノード数 :  21 21
